In [26]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from config import omdb_api_key, tmdb_api_key
import numpy as np

In [10]:
budget_df = pd.read_csv('tn.movie_budgets.csv')

In [11]:
budget_df['production_budget'] = budget_df['production_budget'].map(lambda x: int(x[1:].replace(',', '')))
budget_df['domestic_gross'] = budget_df['domestic_gross'].map(lambda x: int(x[1:].replace(',', '')))
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].map(lambda x: int(x[1:].replace(',', '')))

In [12]:
budget_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null int64
domestic_gross       5782 non-null int64
worldwide_gross      5782 non-null int64
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


In [15]:
budget_df['worldwide_profit'] = budget_df['worldwide_gross'] - budget_df['production_budget']

In [16]:
budget_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,worldwide_profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,234495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400


In [17]:
budget_df.describe()

,id,production_budget,domestic_gross,worldwide_gross,worldwide_profit
count,5782.000000,5.782000e+03,5.782000e+03,5.782000e+03,5.782000e+03
mean,50.372363,3.158776e+07,4.187333e+07,9.148746e+07,5.989970e+07
std,28.821076,4.181208e+07,6.824060e+07,1.747200e+08,1.460889e+08
min,1.000000,1.100000e+03,0.000000e+00,0.000000e+00,-2.002376e+08
25%,25.000000,5.000000e+06,1.429534e+06,4.125415e+06,-2.189071e+06
50%,50.000000,1.700000e+07,1.722594e+07,2.798445e+07,8.550286e+06
75%,75.000000,4.000000e+07,5.234866e+07,9.764584e+07,6.096850e+07
max,100.000000,4.250000e+08,9.366622e+08,2.776345e+09,2.351345e+09


In [61]:
def get_tmdb_info(title):
    answer = None
    response = requests.get(f'https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}')
    try:
        for movie in response.json()['results']:
            if movie['original_title'] == title or movie['title'] == title:
                answer = movie
                break
    except KeyError:
        answer = None
    return answer

In [63]:
results = []
for i, title in enumerate(budget_df.movie):
    print(i, title)
    title_result = get_tmdb_info(title)
    if title_result:
        results.append(title_result)
tmdb_df = pd.DataFrame(results)

0 Avatar
1 Pirates of the Caribbean: On Stranger Tides
2 Dark Phoenix
3 Avengers: Age of Ultron
4 Star Wars Ep. VIII: The Last Jedi
5 Star Wars Ep. VII: The Force Awakens
6 Avengers: Infinity War
7 Pirates of the Caribbean: At Worldâs End
8 Justice League
9 Spectre
10 The Dark Knight Rises
11 Solo: A Star Wars Story
12 The Lone Ranger
13 John Carter
14 Tangled
15 Spider-Man 3
16 Captain America: Civil War
17 Batman v Superman: Dawn of Justice
18 The Hobbit: An Unexpected Journey
19 Harry Potter and the Half-Blood Prince
20 The Hobbit: The Desolation of Smaug
21 The Hobbit: The Battle of the Five Armies
22 The Fate of the Furious
23 Superman Returns
24 Pirates of the Caribbean: Dead Men Tell No Tales
25 Quantum of Solace
26 The Avengers
27 Pirates of the Caribbean: Dead Manâs Chest
28 Man of Steel
29 The Chronicles of Narnia: Prince Caspian
30 The Amazing Spider-Man
31 Battleship
32 Transformers: The Last Knight
33 Jurassic World
34 Men in Black 3
35 Transformers: Revenge of the Fal

350 10,000 B.C.
351 Ice Age: Collision Course
352 Town & Country
353 Kingsman: The Golden Circle
354 Gone in 60 Seconds
355 Gladiator
356 The Hangover 3
357 A Wrinkle in Time
358 Public Enemies
359 Casino Royale
360 Minority Report
361 Bumblebee
362 Cloud Atlas
363 Harry Potter and the Chamber of Secrets
364 Terminator 2: Judgment Day
365 Batman Forever
366 Planet of the Apes
367 National Treasure
368 Django Unchained
369 Little Fockers
370 True Lies
371 American Gangster
372 Cloudy with a Chance of Meatballs
373 The Other Guys
374 Lemony Snicket's A Series of Unfortunate Events
375 The Wolf of Wall Street
376 Epic
377 Eraser
378 The Hunchback of Notre Dame
379 The Emperor's New Groove
380 The Expendables 2
381 Where the Wild Things Are
382 Eragon
383 Hercules
384 The Tourist
385 End of Days
386 The Stepford Wives
387 Surf's Up
388 Blood Diamond
389 Legend of the Guardians: The Owls of Ga'Hoole
390 Starship Troopers
391 Land of the Lost
392 Peter Pan
393 Wonder Park
394 Catwoman
395 Th

747 Basic Instinct 2
748 Agora
749 Despicable Me
750 Dinner for Schmucks
751 Red Sparrow
752 The Twilight Saga: Eclipse
753 The Sum of All Fears
754 Ted 2
755 The Score
756 Hall Pass
757 Money Train
758 Dreamcatcher
759 Mystery Men
760 The Insider
761 Downsizing
762 Doctor Seuss' The Lorax
763 Body of Lies
764 Abraham Lincoln: Vampire Hunter
765 Last Man Standing
766 The Last Legion
767 Entrapment
768 The X Files: Fight the Future
769 Need for Speed
770 Saving Private Ryan
771 What Women Want
772 Lincoln
773 Ice Age
774 Apollo 13
775 The Matrix
776 Hotel Transylvania 3: Summer Vacation
777 Les Miserables
778 The Santa Clause 2
779 Total Recall
780 You've Got Mail
781 Unbroken
782 Spy!
783 Due Date
784 Step Brothers
785 Jumanji
786 The Mask of Zorro
787 Space Cowboys
788 Annie
789 Cliffhanger
790 Peteâs Dragon
791 Broken Arrow
792 World Trade Center
793 The Kid
794 Mona Lisa Smile
795 The Dictator
796 Eyes Wide Shut
797 This Means War
798 Focus
799 Blade: Trinity
800 Red Dawn
801 We A

1155 The Nice Guys
1156 For Love of the Game
1157 Jack Frost
1158 Marmaduke
1159 Striptease
1160 Hereafter
1161 Murder by Numbers
1162 Assassins
1163 Zoolander 2
1164 Defiance
1165 Hannibal Rising
1166 The Story of Us
1167 The Host
1168 Basic
1169 Run All Night
1170 Blood Work
1171 The International
1172 Escape from L.A.
1173 Twisted
1174 The Life Aquatic with Steve Zissou
1175 The Iron Giant
1176 Hellboy
1177 Your Highness
1178 Snowden
1179 Dream House
1180 Free State of Jones
1181 Gamer
1182 Man of the House
1183 The Life of David Gale
1184 Beautiful Creatures
1185 Into the Blue
1186 Eastern Promises
1187 Baby's Day Out
1188 The Messenger: The Story of Joan of Arc
1189 Fair Game
1190 Drive Angry
1191 Mad City
1192 The Scarlet Letter
1193 Domino
1194 Jade
1195 Street Fighter: The Legend of Chun-Li
1196 Death to Smoochy
1197 Early Man
1198 The Boat That Rocked
1199 The Big Bounce
1200 What Planet Are You From?
1201 Child 44
1202 Outlander
1203 Upside Down
1204 Shanghai
1205 Hannibal th

1552 Yi dai zong shi
1553 The Fast and the Furious
1554 Sweet Home Alabama
1555 The Ugly Truth
1556 Spy Kids 2: The Island of Lost Dreams
1557 Star Trek: Generations
1558 Captain Underpants: The First Epic Movie
1559 Paul Blart: Mall Cop 2
1560 Mouse Hunt
1561 Why Him?
1562 Water for Elephants
1563 Life as We Know It
1564 The Hurricane
1565 Riddick
1566 Heartbreakers
1567 Enough
1568 The 5th Wave
1569 Push
1570 Rush
1571 Angel Eyes
1572 Joe Somebody
1573 The Ninth Gate
1574 Only the Brave
1575 Extreme Measures
1576 The Thing
1577 Rock Star
1578 White Squall
1579 City of Ember
1580 Switchback
1581 The Sisters Brothers
1582 Texas Rangers
1583 Virgin Territory
1584 The Upside
1585 What to Expect When You're Expecting
1586 The Master
1587 The Express
1588 Twilight
1589 We're the Millers
1590 Creed
1591 The Town
1592 Dolphin Tale
1593 Game Night
1594 Burn After Reading
1595 13 Going On 30
1596 Nim's Island
1597 How to be Single
1598 Cop Out
1599 The Rite
1600 Space Chimps
1601 Magnolia
1602

...

3895 The Boondock Saints 2: All Saints Day
3896 Prom
3897 The Inkwell
3898 Shadow of the Vampire
3899 Me and Earl and the Dying Girl
3900 It's Kind of a Funny Story
3901 The Pallbearer
3902 Woman on Top
3903 Held Up
3904 Morgan
3905 Anomalisa
3906 CachÃ©
3907 Another Year
3908 8 femmes
3909 Showdown in Little Tokyo
3910 Clay Pigeons
3911 Le passÃ©
3912 Lean on Pete
3913 Made in Dagenham
3914 And When Did You Last See Your Father?
3915 El Chicano
3916 The Secret of Kells
3917 Prefontaine
3918 The Wicked Lady
3919 Down in the Valley
3920 Brooklyn Rules
3921 The Singing Detective
3922 Fido
3923 London Fields
3924 Restless
3925 The Land Girls
3926 The Wendell Baker Story
3927 Wild Target
3928 Pathology
3929 Wuthering Heights
3930 A Street Cat Named Bob
3931 10th & Wolf
3932 Aloft
3933 Fireflies in the Garden
3934 Dear Wendy
3935 Akira
3936 Welcome to Dongmakgol
3937 Feichang Xingyun
3938 Don Gato, el inicio de la pandilla
3939 The Death and Life of Bobby Z
3940 Backstabbing For Beginners
3

4286 The Man From Snowy River
4287 House Party 2
4288 Doug's 1st Movie
4289 Race
4290 Mommie Dearest
4291 Still Alice
4292 Halloween 4: The Return of Michael Myers
4293 Apollo 18
4294 Paul, Apostle of Christ
4295 Addicted
4296 O (Othello)
4297 Halloween: The Curse of Michael Myers
4298 The Egyptian
4299 Eve's Bayou
4300 Nighthawks
4301 Friday the 13th Part VIII: Jason Takes Manhattan
4302 Spring Breakers
4303 It Comes at Night
4304 Y Tu MamÃ¡ TambiÃ©n
4305 Shaun of the Dead
4306 The Haunting of Molly Hartley
4307 Hillaryâs America: The Secret History of the Democratic Party
4308 Lone Star
4309 April Fool's Day
4310 Diner
4311 Before I Fall
4312 Lone Wolf McQuade
4313 Sunshine Cleaning
4314 Fifty Shades of Black
4315 A Farewell To Arms
4316 Solomon and Sheba
4317 The Beastmaster
4318 Not Easily Broken
4319 Momsâ Night Out
4320 The Belko Experiment
4321 The Perfect Match
4322 Digimon: The Movie
4323 Saved!
4324 Les invasions barbares
4325 Robin and Marian
4326 Force 10 from Navarone


4678 Friday the 13th Part VI: Jason Lives
4679 Martin Lawrence Live: RunTelDat
4680 The Apartment
4681 Cat on a Hot Tin Roof
4682 Beneath the Planet of the Apes
4683 Pokemon 3: The Movie
4684 Jason Goes to Hell: The Final Friday
4685 Spaced Invaders
4686 Family Plot
4687 Torn Curtain
4688 Upgrade
4689 Dave Chappelle's Block Party
4690 Krush Groove
4691 Elmer Gantry
4692 Next Day Air
4693 Judgment at Nuremberg
4694 Belly
4695 Trippinâ
4696 Red River
4697 The Deep End
4698 Before Midnight
4699 Teen Wolf Too
4700 The Collector
4701 Phantasm II
4702 Let There Be Light
4703 Phat Girlz
4704 Woman Thou Art Loosed
4705 Real Women Have Curves
4706 Gallipoli
4707 Water
4708 Donât Think Twice
4709 Swiss Army Man
4710 East is East
4711 Whipped
4712 Kama Sutra
4713 Frances Ha
4714 Please Give
4715 Willy Wonka & the Chocolate Factory
4716 Warlock: The Armageddon
4717 8 Heads in a Duffel Bag
4718 Days of Heaven
4719 Thirteen Conversations About One Thing
4720 Compadres
4721 Jawbreaker
4722 Basqui

5055 Out of the Dark
5056 Bubble
5057 Dil Jo Bhi Kahey...
5058 Mississippi Mermaid
5059 Two Night Stand
5060 I Love Your Work
5061 The Kingâs Thief
5062 Insidious
5063 Moonlight
5064 Cabin Fever
5065 Waitress
5066 Kickboxer
5067 Bloodsport
5068 Mr. Smith Goes To Washington
5069 Unsane
5070 Kids
5071 The Squid and the Whale
5072 Kissing Jessica Stein
5073 Spellbound
5074 Dawn of the Dead
5075 Exotica
5076 The To Do List
5077 The Gatekeepers
5078 Buffalo '66
5079 Repo Man
5080 Iâm Not Ashamed
5081 Blood Simple
5082 Nueve Reinas
5083 The Lords of Salem
5084 31
5085 The Ballad of Jack and Rose
5086 Unsullied
5087 Killing Zoe
5088 The Believer
5089 Snow Angels
5090 Session 9
5091 MOOZ-lum
5092 I Want Someone To Eat Cheese With
5093 Amigo
5094 Hatchet
5095 My Name is Bruce
5096 The Disappearance of Alice Creed
5097 Modern Times
5098 The Salon
5099 The Square
5100 Stolen Summer
5101 Road Hard
5102 Forty Shades of Blue
5103 Trucker
5104 Pontypool
5105 Me, You and Five Bucks
5106 Housebound

5440 Groove
5441 The R.M.
5442 Twin Falls Idaho
5443 Mean Creek
5444 Drinking Buddies
5445 Hurricane Streets
5446 Never Again
5447 Shichinin no samurai
5448 Civil Brand
5449 Monsters
5450 Lonesome Jim
5451 Deceptive Practice: The Mysteries and Mentors of Ricky Jay
5452 The Other Dream Team
5453 O Menino e o Mundo
5454 Rubber
5455 Johnny Suede
5456 Finishing the Game
5457 Kiss the Bride
5458 The Journey
5459 Home
5460 The Slaughter Rule
5461 The Living Wake
5462 Scott Walker: 30 Century Man
5463 The Californians
5464 Born to Fly: Elizabeth Streb vs. Gravity
5465 Detention of the Dead
5466 Higher Power
5467 Feng kuang de shi tou
5468 Now is Good
5469 Griffin & Phoenix
5470 Everything Put Together
5471 Drones
5472 Enter Nowhere
5473 Girls Gone Dead
5474 Insomnia Manica
5475 America Is Still The Place
5476 The Outrageous Sophie Tucker
5477 Subconscious
5478 Roadside
5479 Diamond Ruff
5480 Love in the Time of Monsters
5481 El rey de Najayo
5482 Fight to the Finish
5483 Bucky and the Squirre

In [69]:
budget_df.rename(columns={'movie':'title'}, inplace=True)

In [70]:
budget_df

,id,release_date,title,production_budget,domestic_gross,worldwide_gross,worldwide_profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,234495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400


In [71]:
temp_df = budget_df.merge(tmdb_df, on='title', how='left')

In [76]:
temp_df

,id_x,release_date_x,title,production_budget,domestic_gross,worldwide_gross,worldwide_profit,popularity,vote_count,video,poster_path,id_y,adult,backdrop_path,original_language,original_title,genre_ids,vote_average,overview,release_date_y
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,96.642,22179.0,False,/6EiRUJpuoeQPghrs3YNktfnqOVh.jpg,19995.0,False,/bIL7ENqh1egWTxN41sM2W42JqPc.jpg,en,Avatar,"[28, 12, 14, 878]",7.5,"In the 22nd century, a paraplegic Marine is di...",2009-12-10
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,56.206,10308.0,False,/9zxsSzXx2UO3yNoS4j97vs4uOhc.jpg,1865.0,False,/bfTKq6mGHt2CzZlBU31JHF9bMog.jpg,en,Pirates of the Caribbean: On Stranger Tides,"[28, 12, 14]",6.5,Captain Jack Sparrow crosses paths with a woma...,2011-05-14
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650,163.063,3995.0,False,/cCTJPelKGLhALq3r51A9uMonxKj.jpg,320288.0,False,/phxiKFDvPeQj4AbkvJLmuZEieDU.jpg,en,Dark Phoenix,"[28, 12, 878]",6.1,The X-Men face their most formidable and power...,2019-06-05
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963,91.719,16654.0,False,/4ssDuvEDkSArWEdyBl2X5EHvYKU.jpg,99861.0,False,/8i6ZDk97Vyh0jHohMG4vFeFuKJh.jpg,en,Avengers: Age of Ultron,"[28, 12, 878]",7.3,When Tony Stark tries to jumpstart a dormant p...,2015-04-22
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5951,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,3.217,0.0,False,/eNRquvMLPj5lv5cg4YRlJHRwU2b.jpg,536434.0,False,None,en,Red 11,"[878, 53]",0.0,A human lab rat in a sinister medical research...,2019-03-15
5952,79,"Apr 2, 1999",Following,6000,48482,240495,234495,6.150,934.0,False,/lK9LE5D67HNyPlhJcjKAgnRh0OO.jpg,11660.0,False,/gCpl06aiAGUrPrmspeqhdKVQRT1.jpg,en,Following,"[18, 53]",7.2,"Bill, an idle, unemployed aspiring writer, wal...",1999-04-02
5953,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662,0.600,0.0,False,/tERTyfuJteH40r5ZIhb1O8Wz0Bz.jpg,292902.0,False,None,en,Return to the Land of Wonders,[99],0.0,Moving between the political sphere and everyd...,2004-04-27
5954,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,0.600,1.0,False,/rvaQbBiZlq144HBKLgMD16uwiy5.jpg,365441.0,False,None,en,A Plague So Pleasant,[27],7.0,"In the near future, zombies have become a prot...",2013-01-26


In [78]:
tmdb_df.to_csv('tmdb_df.csv', index = False)